In [31]:
import math
import pandas as pd
import altair as alt

In [ ]:
df_rank = pd.read_csv('data/historical_bwf_ranking.csv')
df_rank.drop(columns=['BWF ID'], inplace=True)
df_rank['highest_rank'] = df_rank.min(axis=1)
df_rank.sort_values(by='highest_rank', inplace=True)
df_rank['name'] = df_rank.apply(lambda row: str(row['First name']).capitalize() + ' ' + str(row['Last name']).capitalize(), axis=1)
df_rank.drop(columns=['First name', 'Last name'], inplace=True)
df_rank.drop(df_rank[df_rank['highest_rank'] > 10].index, inplace=True)
col_highest_rank = df_rank.pop("highest_rank")
df_rank.insert(0, col_highest_rank.name, col_highest_rank)
col_name = df_rank.pop("name")
df_rank.insert(0, col_name.name, col_name)
df_rank.reset_index(drop=True, inplace=True)
df_rank.sample(5)

In [ ]:
df_top_10 = pd.DataFrame(columns=['name', 'date', 'rank'])
i_top_10 = 0
for i_rank, row in df_rank.iterrows():
    highest_rank = row['highest_rank']
    if math.isnan(highest_rank) == False and int(highest_rank) <= 5:
        for j in range(2, 498):
            rank = df_rank.iloc[i_rank, j]
            if math.isnan(rank) == False and int(rank) <= 50:
                df_top_10.loc[i_top_10] = [row['name'], df_rank.columns[j], int(rank)]
                i_top_10 += 1

df_top_10.sample(5)

In [34]:
alt.renderers.set_embed_options(theme='light')
alt.data_transformers.disable_max_rows()

selection = alt.selection_multi(fields=['name'], bind='legend')
brush = alt.selection(type='interval', encodings=['x'])

base = alt.Chart(df_top_10).mark_line(interpolate='step-after', point=True).encode(
    x='date:T',
    y=alt.Y('rank', impute=alt.ImputeParams(value=None), scale=alt.Scale(domain=[50,1], padding=0)),
    color='name:N',
    opacity=alt.condition(selection, alt.value(0.9), alt.value(0.05)),
).add_selection(
    selection, brush
)

upper = base.mark_line(interpolate='step-after', point=True, size=2).encode(
    alt.X('date:T', scale=alt.Scale(domain=brush), axis=alt.Axis(grid=False, title=None)),
    tooltip=['name', 'date:T', 'rank'],
).properties(
    width=800,
    height=400
)

lower = base.mark_line(interpolate='step-after', point=True, size=0.5).encode(
    alt.X('date:T', axis=alt.Axis(grid=False)),
    alt.Y(
        'rank',
        axis=None,
        impute=alt.ImputeParams(value=None),
        scale=alt.Scale(domain=[50,1], padding=0),
    )
).properties(
    width=800,
    height=50,
)

alt.vconcat(upper & lower).configure_point(
    size=5
).configure_range(
    category={'scheme': 'tableau10'}
).configure_title(
    fontSize=24,
    anchor='middle'
).properties(
    title='MS Players with History Top 10 BWF World Ranking'
)

alt.VConcatChart(...)